In [1]:
from sagas.nlu.nlu_tools import vis_tree
from sagas.nlu.ruleset_procs import cached_chunks
chunks = cached_chunks('Nosotros estamos en la escuela.',
                       source='es',
                       engine='stanza')
ds=chunks['root_domains'][0]
vis_tree(ds, 'es', trans=True)

root: estamos(estar; we're, verb, 2)
├── obl: escuela(escuela; school, noun, 5)
│   ├── det: la(el; the, det, 4)
│   └── case: en(en; in, adp, 3)
├── nsubj: Nosotros(yo; We, pron, 1)
└── punct: .(_, punct, 6)


In [2]:
chunks['doc'].as_json

[{'index': 1,
  'text': 'Nosotros',
  'lemma': 'yo',
  'upos': 'PRON',
  'xpos': 'PRON',
  'feats': 'Case=Acc,Nom|Gender=Masc|Number=Plur|Person=1|PronType=Prs',
  'governor': 2,
  'dependency_relation': 'nsubj',
  'entity': ['O'],
  'segments': []},
 {'index': 2,
  'text': 'estamos',
  'lemma': 'estar',
  'upos': 'VERB',
  'xpos': 'VERB',
  'feats': 'Mood=Ind|Number=Plur|Person=1|Tense=Pres|VerbForm=Fin',
  'governor': 0,
  'dependency_relation': 'root',
  'entity': ['O'],
  'segments': []},
 {'index': 3,
  'text': 'en',
  'lemma': 'en',
  'upos': 'ADP',
  'xpos': 'ADP',
  'feats': 'AdpType=Prep',
  'governor': 5,
  'dependency_relation': 'case',
  'entity': ['O'],
  'segments': []},
 {'index': 4,
  'text': 'la',
  'lemma': 'el',
  'upos': 'DET',
  'xpos': 'DET',
  'feats': 'Definite=Def|Gender=Fem|Number=Sing|PronType=Art',
  'governor': 5,
  'dependency_relation': 'det',
  'entity': ['O'],
  'segments': []},
 {'index': 5,
  'text': 'escuela',
  'lemma': 'escuela',
  'upos': 'NOUN',


In [15]:
from anytree.node.nodemixin import NodeMixin
from anytree.node.util import _repr
from sagas.nlu.uni_intf import SentenceIntf, WordIntf, RootWordImpl
from sagas.nlu.features import feats_map
class Token(object):
    def __init__(self, tok:WordIntf):
        self.tok=tok        
        self.name=tok.dependency_relation if tok is not None else '_'
class AnalNode(NodeMixin, Token):
    def __init__(self, tok, parent=None, children=None, **kwargs):
        super(AnalNode, self).__init__(tok)
        self.__dict__.update(kwargs)
        if tok:
            self.__dict__.update(tok.ctx)
            self.feats=feats_map(tok.feats)
        self.parent = parent
        if children:
            self.children = children

    def __repr__(self):
        return _repr(self)

words=chunks['doc'].words
root = AnalNode(words[0])
root

AnalNode(dependency_relation='nsubj', entity=['O'], feats={'Case': 'Acc,Nom', 'Gender': 'Masc', 'Number': 'Plur', 'Person': '1', 'PronType': 'Prs'}, governor=2, index=1, lemma='yo', name='nsubj', segments=[], text='Nosotros', tok=<JsonifyWordImpl index=1;text=Nosotros;lemma=yo;upos=PRON;xpos=PRON;feats=Case=Acc,Nom|Gender=Masc|Number=Plur|Person=1|PronType=Prs;governor=2;dependency_relation=nsubj>, upos='PRON', xpos='PRON')

In [16]:
from anytree import Node, RenderTree, AsciiStyle, Walker, Resolver

node_map={word.index:AnalNode(word) for word in words}
node_map[0]=AnalNode(None)
tree_root=next(w for w in node_map.values() if w.governor==0)
def set_parent(w):
    if w.tok:
        w.parent=node_map[w.tok.governor]
list(map(set_parent, node_map.values()))
# print(RenderTree(tree_root, style=AsciiStyle()).by_attr('name'))
print(RenderTree(tree_root, style=AsciiStyle()).by_attr(lambda n: f"{n.dependency_relation}: {n.text}"))

root: estamos
|-- nsubj: Nosotros
|-- obl: escuela
|   |-- case: en
|   +-- det: la
+-- punct: .


In [17]:
from anytree.search import findall, findall_by_attr
words=findall_by_attr(tree_root, name='upos', value='VERB')
words

(AnalNode(dependency_relation='root', entity=['O'], feats={'Mood': 'Ind', 'Number': 'Plur', 'Person': '1', 'Tense': 'Pres', 'VerbForm': 'Fin'}, governor=0, index=2, lemma='estar', name='root', segments=[], text='estamos', tok=<JsonifyWordImpl index=2;text=estamos;lemma=estar;upos=VERB;xpos=VERB;feats=Mood=Ind|Number=Plur|Person=1|Tense=Pres|VerbForm=Fin;governor=0;dependency_relation=root>, upos='VERB', xpos='VERB'),)